In [1]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import os, sys, glob, string, argparse
from climsim_utils.data_utils import *

In [2]:
grid_info = xr.open_dataset('/global/cfs/cdirs/m4334/jerry/climsim3_dev/grid_info/ClimSim_low-res_grid-info.nc')
input_mean = xr.open_dataset('/global/cfs/cdirs/m4334/jerry/climsim3_dev/preprocessing/normalizations/inputs/input_mean_v2_rh_mc_pervar.nc')
input_max = xr.open_dataset('/global/cfs/cdirs/m4334/jerry/climsim3_dev/preprocessing/normalizations/inputs/input_max_v2_rh_mc_pervar.nc')
input_min = xr.open_dataset('/global/cfs/cdirs/m4334/jerry/climsim3_dev/preprocessing/normalizations/inputs/input_min_v2_rh_mc_pervar.nc')
output_scale = xr.open_dataset('/global/cfs/cdirs/m4334/jerry/climsim3_dev/preprocessing/normalizations/outputs/output_scale_std_lowerthred_v6.nc')
lbd_qn = np.loadtxt('/global/cfs/cdirs/m4334/jerry/climsim3_dev/preprocessing/normalizations/inputs/qn_exp_lambda_large.txt', delimiter = ',')

data = data_utils(grid_info = grid_info,
                input_mean = input_mean, 
                input_max = input_max, 
                input_min = input_min, 
                output_scale = output_scale,
                qinput_log = False,
                normalize = False)
data.set_to_v6_vars()

lat = grid_info['lat'].values
lon = grid_info['lon'].values
lon = grid_info['lon'].values
lon = ((lon + 180) % 360) - 180
level = grid_info.lev.values
lat_bin_mids = data.lat_bin_mids

color_dict = {
    'unet': 'green',
    'squeezeformer': 'purple',
    'pure_resLSTM': 'blue',
    'pao_model': 'red',
    'convnext': 'gold',
    'encdec_lstm': 'orange',
}

label_dict = {
    'unet': 'U-Net',
    'squeezeformer': 'Squeezeformer',
    'pure_resLSTM': 'Pure ResLSTM',
    'pao_model': 'Pao Model',
    'convnext': 'ConvNeXt',
    'encdec_lstm': 'Encoder-Decoder LSTM',
}

# retrieve precomputed 1-year tropopause level distribution (we used as a microphysics constraint)
idx_p400_t10 = np.load('/pscratch/sd/z/zeyuanhu/hu_etal2024_data/microphysics_hourly/first_true_indices_p400_t10.npy')
for i in range(idx_p400_t10.shape[0]):
    for j in range(idx_p400_t10.shape[1]):
        idx_p400_t10[i,j] = level[int(idx_p400_t10[i,j])]

idx_p400_t10 = idx_p400_t10.mean(axis=0)
idx_p400_t10 = idx_p400_t10[np.newaxis,:]

idx_tropopause_zm = data.zonal_bin_weight_2d(idx_p400_t10)

def area_mean(ds, var):
    arr = ds[var].values
    arr_reshaped = np.transpose(arr, (0,2,1))
    arr_zonal_mean = data.zonal_bin_weight_3d(arr_reshaped)
    return arr_zonal_mean

def zonal_diff(ds_sp, ds_nn, var):
    diff_zonal_mean = (area_mean(ds_nn, var) - area_mean(ds_sp, var)).mean(axis = 0)
    diff_zonal = xr.DataArray(diff_zonal_mean.T, dims = ['level', 'lat'], coords = {'level':level, 'lat': lat_bin_mids})
    return diff_zonal

def get_dp(ds):
    ps = ds['PS']
    p_interface = (ds['hyai'] * ds['P0'] + ds['hybi'] * ds['PS']).values
    if p_interface.shape[0] == 61:
        p_interface = np.swapaxes(p_interface, 0, 1)
    dp = p_interface[:,1:61,:] - p_interface[:,0:60,:]
    return dp

def get_tcp_mean(ds, area_weight):
    cld = (ds['CLDICE'][:,:,:] + ds['CLDLIQ'][:,:,:]).values
    dp = get_dp(ds)
    tcp = np.sum(cld*dp, axis = 1)/9.81
    tcp_mean = np.average(tcp, weights = area_weight, axis = 1)
    return tcp_mean

grid_area = grid_info['area'].values
lev = grid_info['lev'].values
area_weight = grid_area/np.sum(grid_area)

land_frac = ds_sp_1['LANDFRAC'][2].values
ocean_frac = ds_sp_1['OCNFRAC'][2].values
land_area = data.grid_info_area * land_frac
ocean_area = data.grid_info_area * ocean_frac
data.lat_bin_land_area_sums = np.array([np.sum(land_area[data.lat_bin_dict[lat_bin]]) for lat_bin in data.lat_bin_dict.keys()])
data.lat_bin_land_area_divs = np.array([1/data.lat_bin_land_area_sums[bin_index] for bin_index in data.lat_bin_indices])
data.lat_bin_ocean_area_sums = np.array([np.sum(ocean_area[data.lat_bin_dict[lat_bin]]) for lat_bin in data.lat_bin_dict.keys()])
data.lat_bin_ocean_area_divs = np.array([1/data.lat_bin_ocean_area_sums[bin_index] for bin_index in data.lat_bin_indices])
data.lat_bin_land_area_divs[np.isnan(data.lat_bin_land_area_divs)] = 1
data.lat_bin_ocean_area_divs[np.isnan(data.lat_bin_ocean_area_divs)] = 1

ds_sp_1 = xr.open_mfdataset('/pscratch/sd/z/zeyuanhu/hu_etal2024_data_v2/data/h0/5year/mmf_ref/control_fullysp_jan_wmlio_r3.eam.h0.000[34567]*.nc')
ds_sp_2 = xr.open_mfdataset('/pscratch/sd/z/zeyuanhu/hu_etal2024_data_v2/data/h0/5year/mmf_b/control_fullysp_jan_wmlio_r3_b.eam.h0.000[34567]*.nc')

ds_sp_1['DQnPHYS'] = ds_sp_1['DQ2PHYS'] + ds_sp_1['DQ3PHYS']
ds_sp_2['DQnPHYS'] = ds_sp_2['DQ2PHYS'] + ds_sp_2['DQ3PHYS']

sp_dp = get_dp(ds_sp_1)
total_weight = sp_dp * area_weight[None, None, :]
total_weight = total_weight.mean(axis = 0)
total_weight = total_weight/total_weight.sum()

In [3]:
# standard_path = '/pscratch/sd/j/jerrylin/hugging/E3SM-MMF_ne4/online_runs/climsim3_ensembles_good/standard/five_year_runs/'

# ds_unet_seed_7 = xr.open_mfdataset(os.path.join(standard_path, 'unet_seed_7/run/unet_seed_7.eam.h0.000[34567]*.nc'))
# ds_unet_seed_43 = xr.open_mfdataset(os.path.join(standard_path, 'unet_seed_43/run/unet_seed_43.eam.h0.000[34567]*.nc'))
# ds_unet_seed_1024 = xr.open_mfdataset(os.path.join(standard_path, 'unet_seed_1024/run/unet_seed_1024.eam.h0.000[34567]*.nc'))

# ds_unet_seed_7['DQnPHYS'] = ds_unet_seed_7['DQ2PHYS'] + ds_unet_seed_7['DQ3PHYS']
# ds_unet_seed_43['DQnPHYS'] = ds_unet_seed_43['DQ2PHYS'] + ds_unet_seed_43['DQ3PHYS']
# ds_unet_seed_1024['DQnPHYS'] = ds_unet_seed_1024['DQ2PHYS'] + ds_unet_seed_1024['DQ3PHYS']

# ds_squeezeformer_seed_7 = xr.open_mfdataset(os.path.join(standard_path, 'squeezeformer_seed_7/run/squeezeformer_seed_7.eam.h0.000[34567]*.nc'))
# ds_squeezeformer_seed_43 = xr.open_mfdataset(os.path.join(standard_path, 'squeezeformer_seed_43/run/squeezeformer_seed_43.eam.h0.000[34567]*.nc'))
# ds_squeezeformer_seed_1024 = xr.open_mfdataset(os.path.join(standard_path, 'squeezeformer_seed_1024/run/squeezeformer_seed_1024.eam.h0.000[34567]*.nc'))

# ds_squeezeformer_seed_7['DQnPHYS'] = ds_squeezeformer_seed_7['DQ2PHYS'] + ds_squeezeformer_seed_7['DQ3PHYS']
# ds_squeezeformer_seed_43['DQnPHYS'] = ds_squeezeformer_seed_43['DQ2PHYS'] + ds_squeezeformer_seed_43['DQ3PHYS']
# ds_squeezeformer_seed_1024['DQnPHYS'] = ds_squeezeformer_seed_1024['DQ2PHYS'] + ds_squeezeformer_seed_1024['DQ3PHYS']

In [4]:
# def area_time_mean_3d(ds, var):
#     arr = ds[var].values[1:,:,:]
#     arr_reshaped = np.transpose(arr, (0,2,1))
#     arr_zonal_mean = data.zonal_bin_weight_3d(arr_reshaped)
#     arr_zonal_time_mean = arr_zonal_mean.mean(axis = 0)
#     arr_zonal_time_mean = xr.DataArray(arr_zonal_time_mean.T, dims = ['hybrid pressure (hPa)', 'latitude'], coords = {'hybrid pressure (hPa)':level, 'latitude': lat_bin_mids})
#     return arr_zonal_time_mean

# def plot_zonal_mean_bias(var):
#     # Create a figure with subplots
#     fig, axs = plt.subplots(2, 3, figsize=(14, 6.5), constrained_layout = True) 
#     # Generate the panel labels
#     labels = [f"({letter})" for letter in string.ascii_lowercase[:6]]
#     latitude_ticks = [-60, -30, 0, 30, 60]
#     latitude_labels = ['60S', '30S', '0', '30N', '60N']
#     # Loop through each variable and its corresponding subplot row
#     var_settings = {
#         'T': {'var_title': 'T', 'scaling': 1.0, 'unit': 'K', 'vmax': 5, 'vmin': -5},
#         'Q': {'var_title': 'Q', 'scaling': 1000.0, 'unit': 'g/kg', 'vmax': 1, 'vmin': -1},
#         'U': {'var_title': 'U', 'scaling': 1.0, 'unit': 'm/s', 'vmax': 4, 'vmin': -4},
#         'CLDLIQ': {'var_title': 'Liquid Cloud', 'scaling': 1e6, 'unit': 'mg/kg', 'vmax': 40, 'vmin': -40},
#         'CLDICE': {'var_title': 'Ice Cloud', 'scaling': 1e6, 'unit': 'mg/kg', 'vmax': 5, 'vmin': -5},
#         'DTPHYS': {'var_title': 'dT/dt', 'scaling': 1., 'unit': 'K/s', 'vmax': 1.5e-5, 'vmin': -1.5e-5},
#         'DQ1PHYS': {'var_title': 'dQ/dt', 'scaling': 1e3, 'unit': 'g/kg/s', 'vmax': 1.2e-5, 'vmin': -1.2e-5},
#         'DUPHYS': {'var_title': 'dU/dt', 'scaling': 1., 'unit': 'm/s²', 'vmax': 2.2e-6, 'vmin': -2.2e-6},
#         'DQnPHYS': {'var_title': 'dQn/dt', 'scaling': 1e6, 'unit': 'mg/kg/s', 'vmax': .0015, 'vmin': -.0015},
#     }

#     unet_seed_7_diff = var_settings[var]['scaling'] * (area_time_mean_3d(ds_unet_seed_7, var) - area_time_mean_3d(ds_sp_1, var))
#     unet_seed_43_diff = var_settings[var]['scaling'] * (area_time_mean_3d(ds_unet_seed_43, var) - area_time_mean_3d(ds_sp_1, var))
#     unet_seed_1024_diff = var_settings[var]['scaling'] * (area_time_mean_3d(ds_unet_seed_1024, var) - area_time_mean_3d(ds_sp_1, var))
#     squeezeformer_seed_7_diff = var_settings[var]['scaling'] * (area_time_mean_3d(ds_squeezeformer_seed_7, var) - area_time_mean_3d(ds_sp_1, var))
#     squeezeformer_seed_43_diff = var_settings[var]['scaling'] * (area_time_mean_3d(ds_squeezeformer_seed_43, var) - area_time_mean_3d(ds_sp_1, var))
#     squeezeformer_seed_1024_diff = var_settings[var]['scaling'] * (area_time_mean_3d(ds_squeezeformer_seed_1024, var) - area_time_mean_3d(ds_sp_1, var))

#     unet_seed_7_diff.plot(ax=axs[0, 0], add_colorbar=False, cmap='RdBu_r', vmin=var_settings[var]['vmin'], vmax=var_settings[var]['vmax'])
#     axs[0, 0].set_title(f'{labels[0]} U-Net (seed 7)')
#     axs[0, 0].invert_yaxis()
#     axs[0, 0].set_xlabel('')

#     unet_seed_43_diff.plot(ax=axs[0, 1], add_colorbar=False, cmap='RdBu_r', vmin=var_settings[var]['vmin'], vmax=var_settings[var]['vmax'])
#     axs[0, 1].set_title(f'{labels[1]} U-Net (seed 43)')
#     axs[0, 1].invert_yaxis()
#     axs[0, 1].set_xlabel('')
#     axs[0, 1].set_ylabel('')

#     unet_seed_1024_diff.plot(ax=axs[0, 2], add_colorbar=False, cmap='RdBu_r', vmin=var_settings[var]['vmin'], vmax=var_settings[var]['vmax'])
#     axs[0, 2].set_title(f'{labels[2]} U-Net (seed 1024)')
#     axs[0, 2].invert_yaxis()
#     axs[0, 2].set_xlabel('')
#     axs[0, 2].set_ylabel('')

#     squeezeformer_seed_7_diff.plot(ax=axs[1, 0], add_colorbar=False, cmap='RdBu_r', vmin=var_settings[var]['vmin'], vmax=var_settings[var]['vmax'])
#     axs[1, 0].set_title(f'{labels[3]} Squeezeformer (seed 7)')
#     axs[1, 0].invert_yaxis()
#     axs[1, 0].set_xlabel('Latitude')

#     squeezeformer_seed_43_diff.plot(ax=axs[1, 1], add_colorbar=False, cmap='RdBu_r', vmin=var_settings[var]['vmin'], vmax=var_settings[var]['vmax'])
#     axs[1, 1].set_title(f'{labels[4]} Squeezeformer (seed 43)')
#     axs[1, 1].invert_yaxis()
#     axs[1, 1].set_xlabel('Latitude')
#     axs[1, 1].set_ylabel('')

#     # encdec_lstm_diff.plot(ax=axs[1, 2], add_colorbar=False, cmap='RdBu_r', vmin=vmin, vmax=vmax)
#     img = squeezeformer_seed_1024_diff.plot(ax=axs[1, 2], add_colorbar=False, cmap='RdBu_r', vmin=var_settings[var]['vmin'], vmax=var_settings[var]['vmax'])
#     axs[1, 2].set_title(f'{labels[5]} Squeezeformer (seed 1024)')
#     axs[1, 2].invert_yaxis()
#     axs[1, 2].set_xlabel('Latitude')
#     axs[1, 2].set_ylabel('')

#     if var == 'CLDICE':
#         tropopause_pressure = idx_tropopause_zm.flatten()  # Flatten to 1D array
#         tropopause_latitude = data.lat_bin_mids  # Latitude values
#         axs[0, 0].plot(tropopause_latitude, tropopause_pressure, 'k--')
#         axs[0, 1].plot(tropopause_latitude, tropopause_pressure, 'k--')
#         axs[0, 2].plot(tropopause_latitude, tropopause_pressure, 'k--')
#         axs[1, 0].plot(tropopause_latitude, tropopause_pressure, 'k--')
#         axs[1, 1].plot(tropopause_latitude, tropopause_pressure, 'k--')
#         axs[1, 2].plot(tropopause_latitude, tropopause_pressure, 'k--')
    
#     plt.tight_layout()
#     # Add a single colorbar to the right of the figure
#     cbar_ax = fig.add_axes([0.92, 0.15, 0.02, 0.7])  # [left, bottom, width, height]
#     cbar = fig.colorbar(img, cax=cbar_ax)
#     cbar.set_label(f'{var_settings[var]["var_title"]} Bias ({var_settings[var]["unit"]})')
#     # Set these ticks and labels for each subplot
#     for ax_row in axs:
#         for ax in ax_row:
#             ax.set_xticks(latitude_ticks)  # Set the positions for the ticks
#             ax.set_xticklabels(latitude_labels)  # Set the custom text labels

#     plt.suptitle(f'5 year zonal mean difference {var_settings[var]["var_title"]} ({var_settings[var]["unit"]})')
#     plt.subplots_adjust(right=0.9, top=0.9)

#     plt.show()

In [5]:
# plot_zonal_mean_bias('T')

In [6]:
# plot_zonal_mean_bias('Q')

In [7]:
# plot_zonal_mean_bias('U')

In [8]:
# plot_zonal_mean_bias('CLDLIQ')

In [9]:
# plot_zonal_mean_bias('CLDICE')

In [10]:
# plot_zonal_mean_bias('DTPHYS')

In [11]:
# plot_zonal_mean_bias('DQ1PHYS')

In [12]:
# plot_zonal_mean_bias('DUPHYS')

In [13]:
# plot_zonal_mean_bias('DQnPHYS')

In [14]:
# months = np.arange(1, 61)
# def calculate_rmse(ds1, ds2, total_weight, var='T'):
#     total_weight = get_dp(ds2) * ds2['area'].values[:,None,:]
#     # Determine the number of months in ds1
#     if var == 'qn':
#         num_months = ds1['CLDLIQ'].shape[0]
#     else:
#         num_months = ds1[var].shape[0]
    
#     # Slice total_weight to match the number of months in ds1
#     total_weight_sliced = total_weight[:num_months, :, :]
    
#     # Initialize the RMSE array with NaN values
#     rmse_per_month = np.full(60, np.nan)
    
#     # Compute RMSE for existing months
#     if var == 'qn':
#         squared_diff = (ds1['CLDLIQ'] + ds1['CLDICE'] - ds2['CLDLIQ'] - ds2['CLDICE'])**2
#     else:
#         squared_diff = (ds1[var] - ds2[var]) ** 2
#     weighted_squared_diff = squared_diff * total_weight_sliced
#     weighted_sum = weighted_squared_diff.sum(axis=(1, 2))
#     total_weight_sum = total_weight_sliced.sum(axis=(1, 2))
#     weighted_mean_squared_diff = weighted_sum / total_weight_sum
#     rmse_existing_months = np.sqrt(weighted_mean_squared_diff)
    
#     # Fill in the RMSE array with the computed values
#     rmse_per_month[:num_months] = rmse_existing_months.values
#     if var == 'Q':
#         rmse_per_month = rmse_per_month * 1e3
#     elif var == 'qn':
#         rmse_per_month = rmse_per_month * 1e6
#     return rmse_per_month


# fig, axes = plt.subplots(2, 2, figsize=(18, 8))

# axes[0,0].plot(months, calculate_rmse(ds_sp_2, ds_sp_1, total_weight, var='T'), label='Internal Variability', color='black', linestyle='--', marker='^', markersize = 4)
# axes[0,0].plot(months, calculate_rmse(ds_unet_seed_7, ds_sp_1, total_weight, var='T'), label = 'U-Net (seed 7)', color = 'green', linestyle='--', marker='o', alpha = .4, markersize = 2)
# axes[0,0].plot(months, calculate_rmse(ds_unet_seed_43, ds_sp_1, total_weight, var='T'), label = 'U-Net (seed 43)', color = 'green', linestyle='--', marker='o', alpha = .4, markersize = 2)
# axes[0,0].plot(months, calculate_rmse(ds_unet_seed_1024, ds_sp_1, total_weight, var='T'), label = 'U-Net (seed 1024)', color = 'green', linestyle='--', marker='o', alpha = .4, markersize = 2)
# axes[0,0].plot(months, calculate_rmse(ds_squeezeformer_seed_7, ds_sp_1, total_weight, var='T'), label = 'Squeezeformer (seed 7)', color = 'purple', linestyle='--', marker='o', alpha = .4, markersize = 2)
# axes[0,0].plot(months, calculate_rmse(ds_squeezeformer_seed_43, ds_sp_1, total_weight, var='T'), label = 'Squeezeformer (seed 43)', color = 'purple', linestyle='--', marker='o', alpha = .4, markersize = 2)
# axes[0,0].plot(months, calculate_rmse(ds_squeezeformer_seed_1024, ds_sp_1, total_weight, var='T'), label = 'Squeezeformer (seed 1024)', color = 'purple', linestyle='--', marker='o', alpha = .4, markersize = 2)

# axes[0,0].set_yscale('log')
# axes[0,0].set_yticks([0.5, 1, 2, 5, 10, 20, 50, 100])
# axes[0,0].set_yticklabels(['0.5', '1', '2', '5', '10', '20', '50', '100'], fontsize=12)
# axes[0,0].set_ylabel('Online RMSE (K)', fontsize=15)
# axes[0,0].set_title('(a) Temperature', fontsize=15)
# axes[0,0].legend(fontsize=13, loc='upper left', ncol = 2)
# axes[0,0].grid(True)

# axes[0,1].plot(months, calculate_rmse(ds_sp_2, ds_sp_1, total_weight, var='Q'), label='Internal Variability', color='black', linestyle='--', marker='^', markersize = 4)
# axes[0,1].plot(months, calculate_rmse(ds_unet_seed_7, ds_sp_1, total_weight, var='Q'), label = 'U-Net (seed 7)', color = 'green', linestyle='--', marker='o', alpha = .4, markersize = 2)
# axes[0,1].plot(months, calculate_rmse(ds_unet_seed_43, ds_sp_1, total_weight, var='Q'), label = 'U-Net (seed 43)', color = 'green', linestyle='--', marker='o', alpha = .4, markersize = 2)
# axes[0,1].plot(months, calculate_rmse(ds_unet_seed_1024, ds_sp_1, total_weight, var='Q'), label = 'U-Net (seed 1024)', color = 'green', linestyle='--', marker='o', alpha = .4, markersize = 2)
# axes[0,1].plot(months, calculate_rmse(ds_squeezeformer_seed_7, ds_sp_1, total_weight, var='Q'), label = 'Squeezeformer (seed 7)', color = 'purple', linestyle='--', marker='o', alpha = .4, markersize = 2)
# axes[0,1].plot(months, calculate_rmse(ds_squeezeformer_seed_43, ds_sp_1, total_weight, var='Q'), label = 'Squeezeformer (seed 43)', color = 'purple', linestyle='--', marker='o', alpha = .4, markersize = 2)
# axes[0,1].plot(months, calculate_rmse(ds_squeezeformer_seed_1024, ds_sp_1, total_weight, var='Q'), label = 'Squeezeformer (seed 1024)', color = 'purple', linestyle='--', marker='o', alpha = .4, markersize = 2)
# axes[0,1].set_yscale('log')
# axes[0,1].set_yticks([0.1, 0.2, 0.5, 1, 2, 5, 10])
# axes[0,1].set_yticklabels(['0.1', '0.2', '0.5', '1', '2', '5', '10'], fontsize=12)
# axes[0,1].set_ylabel('Online RMSE (g/kg)', fontsize=15)
# axes[0,1].set_title('(b) Moisture', fontsize=15)
# axes[0,1].grid(True)

# axes[1,0].plot(months, calculate_rmse(ds_sp_2, ds_sp_1, total_weight, var='U'), label='Internal Variability', color='black', linestyle='--', marker='^', markersize = 4)
# axes[1,0].plot(months, calculate_rmse(ds_unet_seed_7, ds_sp_1, total_weight, var='U'), label = 'U-Net (seed 7)', color = 'green', linestyle='--', marker='o', alpha = .4, markersize = 2)
# axes[1,0].plot(months, calculate_rmse(ds_unet_seed_43, ds_sp_1, total_weight, var='U'), label = 'U-Net (seed 43)', color = 'green', linestyle='--', marker='o', alpha = .4, markersize = 2)
# axes[1,0].plot(months, calculate_rmse(ds_unet_seed_1024, ds_sp_1, total_weight, var='U'), label = 'U-Net (seed 1024)', color = 'green', linestyle='--', marker='o', alpha = .4, markersize = 2)
# axes[1,0].plot(months, calculate_rmse(ds_squeezeformer_seed_7, ds_sp_1, total_weight, var='U'), label = 'Squeezeformer (seed 7)', color = 'purple', linestyle='--', marker='o', alpha = .4, markersize = 2)
# axes[1,0].plot(months, calculate_rmse(ds_squeezeformer_seed_43, ds_sp_1, total_weight, var='U'), label = 'Squeezeformer (seed 43)', color = 'purple', linestyle='--', marker='o', alpha = .4, markersize = 2)
# axes[1,0].plot(months, calculate_rmse(ds_squeezeformer_seed_1024, ds_sp_1, total_weight, var='U'), label = 'Squeezeformer (seed 1024)', color = 'purple', linestyle='--', marker='o', alpha = .4, markersize = 2)
# axes[1,0].set_yscale('log')
# axes[1,0].set_yticks([0.5, 1, 2, 5, 10, 20, 50, 100])
# axes[1,0].set_yticklabels(['0.5', '1', '2', '5', '10', '20', '50', '100'], fontsize=20)
# axes[1,0].set_ylabel('Online RMSE (m/s)', fontsize=15)
# axes[1,0].set_title('(c) Zonal Wind', fontsize=15)
# axes[1,0].grid(True)

# axes[1,1].plot(months, calculate_rmse(ds_sp_2, ds_sp_1, total_weight, var='qn'), label='Internal Variability', color='black', linestyle='--', marker='^', markersize = 4)
# axes[1,1].plot(months, calculate_rmse(ds_unet_seed_7, ds_sp_1, total_weight, var='qn'), label = 'U-Net (seed 7)', color = 'green', linestyle='--', marker='o', alpha = .4, markersize = 2)
# axes[1,1].plot(months, calculate_rmse(ds_unet_seed_43, ds_sp_1, total_weight, var='qn'), label = 'U-Net (seed 43)', color = 'green', linestyle='--', marker='o', alpha = .4, markersize = 2)
# axes[1,1].plot(months, calculate_rmse(ds_unet_seed_1024, ds_sp_1, total_weight, var='qn'), label = 'U-Net (seed 1024)', color = 'green', linestyle='--', marker='o', alpha = .4, markersize = 2)
# axes[1,1].plot(months, calculate_rmse(ds_squeezeformer_seed_7, ds_sp_1, total_weight, var='qn'), label = 'Squeezeformer (seed 7)', color = 'purple', linestyle='--', marker='o', alpha = .4, markersize = 2)
# axes[1,1].plot(months, calculate_rmse(ds_squeezeformer_seed_43, ds_sp_1, total_weight, var='qn'), label = 'Squeezeformer (seed 43)', color = 'purple', linestyle='--', marker='o', alpha = .4, markersize = 2)
# axes[1,1].plot(months, calculate_rmse(ds_squeezeformer_seed_1024, ds_sp_1, total_weight, var='qn'), label = 'Squeezeformer (seed 1024)', color = 'purple', linestyle='--', marker='o', alpha = .4, markersize = 2)
# axes[1,1].set_yscale('log')
# axes[1,1].set_yticks([2, 5, 10, 20, 50, 100, 200, 500])
# axes[1,1].set_yticklabels(['2', '5', '10', '20', '50', '100', '200', '500'], fontsize=12)
# axes[1,1].set_ylabel('Online RMSE (mg/kg)', fontsize=15)
# axes[1,1].set_title('(d) Total Cloud', fontsize=15)
# axes[1,1].grid(True)

# axes[1,0].set_xlabel('Month', fontsize=15)
# axes[1,1].set_xlabel('Month', fontsize=15)

# fig.suptitle('Five Year Online Root Mean Squared Error (Standard)', fontsize = 16)

In [15]:
# xtime = np.arange(60)

# area_weight_nh = np.where(lat > 30, area_weight, 0)
# area_weight_sh = np.where(lat < -30, area_weight, 0)
# area_weight_tropics = np.where((lat > - 30) & (lat < 30), area_weight, 0)

# # Titles for columns
# column_titles = ['Global mean', '30N-90N mean', '30S-90S mean', '30S-30N mean']

# # Y-axis labels for rows
# row_ylabels = [
#     'T$_{59}$ (K)',
#     'Precipitable water (kg/m$^2$)',
#     'Total cloud path (kg/m$^2$)'
# ]

# # Define area weights
# weights = ['area_weight', 'area_weight_nh', 'area_weight_sh', 'area_weight_tropics']
# weights_dict = {
#     'area_weight': area_weight,
#     'area_weight_nh': area_weight_nh,
#     'area_weight_sh': area_weight_sh,
#     'area_weight_tropics': area_weight_tropics,
# }

# # Define variables and associated averaging functions
# variables = ['T', 'TMQ', 'TCP']
# get_mean_function = {
#     'T': lambda ds, w: np.average(ds['T'][:, -1, :].values, weights=w, axis=1),
#     'TMQ': lambda ds, w: np.average(ds['TMQ'][:, :].values, weights=w, axis=1),
#     'TCP': lambda ds, w: get_tcp_mean(ds, w)
# }

# # Create a figure with 3 rows and 4 columns
# fig, axes = plt.subplots(3, 4, figsize=(18, 8))

# # Generate subplots
# for row, var in enumerate(variables):
#     for col, weight_key in enumerate(weights):
#         weight = weights_dict[weight_key]
#         ds_sp_1_mean = get_mean_function[var](ds_sp_1, weight)
#         ds_sp_2_mean = get_mean_function[var](ds_sp_2, weight)
#         unet_seed_7_mean = get_mean_function[var](ds_unet_seed_7, weight)
#         unet_seed_43_mean = get_mean_function[var](ds_unet_seed_43, weight)
#         unet_seed_1024_mean = get_mean_function[var](ds_unet_seed_1024, weight)
#         squeezeformer_seed_7_mean = get_mean_function[var](ds_squeezeformer_seed_7, weight)
#         squeezeformer_seed_43_mean = get_mean_function[var](ds_squeezeformer_seed_43, weight)
#         squeezeformer_seed_1024_mean = get_mean_function[var](ds_squeezeformer_seed_1024, weight)
        
#         line_mmf, = axes[row, col].plot(xtime, ds_sp_1_mean, label='MMF', color='black')
#         line_mmf2, = axes[row, col].plot(xtime, ds_sp_2_mean, label='MMF2', color='black', linestyle = 'dashed')
#         line_unet_seed_7, = axes[row, col].plot(xtime, unet_seed_7_mean, label='U-Net (seed 7)', color='green', linestyle = ':')
#         line_unet_seed_43, = axes[row, col].plot(xtime, unet_seed_43_mean, label='U-Net (seed 43)', color='green', linestyle = ':')
#         line_unet_seed_1024, = axes[row, col].plot(xtime, unet_seed_1024_mean, label='U-Net (seed 1024)', color='green', linestyle = ':')
#         line_squeezeformer_seed_7, = axes[row, col].plot(xtime, squeezeformer_seed_7_mean, label='Squeezeformer (seed 7)', color='purple', linestyle = ':')
#         line_squeezeformer_seed_43, = axes[row, col].plot(xtime, squeezeformer_seed_43_mean, label='Squeezeformer (seed 43)', color='purple', linestyle = ':')
#         line_squeezeformer_seed_1024, = axes[row, col].plot(xtime, squeezeformer_seed_1024_mean, label='Squeezeformer (seed 1024)', color='purple', linestyle = ':')

#         axes[row, col].tick_params(axis='both', labelsize=12) 
#         # Set column titles
#         if row == 0:
#             axes[row, col].set_title(column_titles[col],fontsize=14)
        
#         # Set row y-labels
#         if col == 0:
#             axes[row, col].set_ylabel(row_ylabels[row],fontsize=12)
        
#         # Set x-label for the last row
#         if row == 2:
#             axes[row, col].set_xlabel("Months",fontsize=12)
        
#         # Add legend to the first row, last column
#         if row == 2 and col == 0:
#             axes[row, col].legend(handles = [line_mmf, line_mmf2, line_unet_seed_7, line_squeezeformer_seed_43],
#                                   labels = ['MMF', 'MMF2', 'U-Net', 'Squeezeformer'], fontsize=10,loc='upper left')
#         # elif row == 2 and col == 1:
#         #     axes[row, col].legend(handles = [line_pure_resLSTM, line_pao_model, line_convnext, line_encdec_lstm],
#         #                           labels = ['Pure ResLSTM', 'Pao Model', 'ConvNeXt', 'Encoder-Decoder LSTM'], fontsize=10,loc='upper left')

# fig.suptitle('Five Year Area-Weighted Mean Values (Standard)', fontsize = 16)
# # Adjust layout and display the plot
# plt.tight_layout()
# # plt.savefig('time_series_mean_5years.pdf', format='pdf', dpi=400, bbox_inches='tight')
# plt.show()

In [16]:
# variables = ['T', 'Q', 'CLDLIQ', 'CLDICE', 'U', 'V']
# scaling_dict = {'T': 1.0, 'Q': 1000.0, 'CLDLIQ': 1e6, 'CLDICE': 1e6, 'U': 1.0, 'V': 1.0}  # Scaling factors
# units_dict = {'T': 'K', 'Q': 'g/kg', 'CLDLIQ': 'mg/kg', 'CLDICE': 'mg/kg', 'U': 'm/s', 'V': 'm/s'}  # Units
# titles = [
#     '(a) T',
#     '(b) Q',
#     '(c) Liquid cloud',
#     '(d) Ice cloud',
#     '(e) U',
#     '(f) V',
# ]  # Main titles with subplot labels

# fig, axes = plt.subplots(2, 3, figsize=(8, 7), sharey=True, constrained_layout=True)  # 2 rows, 3 columns
# axes = axes.flatten()  # Flatten axes for easier iteration

# for ax, var, title in zip(axes, variables, titles):
#     # Use specified scaling for the variable
#     scaling = scaling_dict[var]

#     # Compute time-mean and RMSE for each variable
#     ds_sp_1_mean = ds_sp_1[var].mean(dim = 'time').values * scaling
#     ds_sp_2_mean = ds_sp_2[var].mean(dim = 'time').values * scaling
#     ds_unet_seed_7_mean = ds_unet_seed_7[var].mean(dim = 'time').values * scaling
#     ds_unet_seed_43_mean = ds_unet_seed_43[var].mean(dim = 'time').values * scaling
#     ds_unet_seed_1024_mean = ds_unet_seed_1024[var].mean(dim = 'time').values * scaling
#     ds_squeezeformer_seed_7_mean = ds_squeezeformer_seed_7[var].mean(dim = 'time').values * scaling
#     ds_squeezeformer_seed_43_mean = ds_squeezeformer_seed_43[var].mean(dim = 'time').values * scaling
#     ds_squeezeformer_seed_1024_mean = ds_squeezeformer_seed_1024[var].mean(dim = 'time').values * scaling

#     mmf_rmse = np.sqrt(np.average((ds_sp_2_mean - ds_sp_1_mean) ** 2, axis = 1, weights = area_weight))
#     unet_seed_7_rmse = np.sqrt(np.average((ds_unet_seed_7_mean - ds_sp_1_mean) ** 2, axis = 1, weights = area_weight))
#     unet_seed_43_rmse = np.sqrt(np.average((ds_unet_seed_43_mean - ds_sp_1_mean) ** 2, axis = 1, weights = area_weight))
#     unet_seed_1024_rmse = np.sqrt(np.average((ds_unet_seed_1024_mean - ds_sp_1_mean) ** 2, axis = 1, weights = area_weight))
#     squeezeformer_seed_7_rmse = np.sqrt(np.average((ds_squeezeformer_seed_7_mean - ds_sp_1_mean) ** 2, axis = 1, weights = area_weight))
#     squeezeformer_seed_43_rmse = np.sqrt(np.average((ds_squeezeformer_seed_43_mean - ds_sp_1_mean) ** 2, axis = 1, weights = area_weight))
#     squeezeformer_seed_1024_rmse = np.sqrt(np.average((ds_squeezeformer_seed_1024_mean - ds_sp_1_mean) ** 2, axis = 1, weights = area_weight))

#     mmf_rmse_global = np.sqrt(np.average((ds_sp_2_mean - ds_sp_1_mean) ** 2, weights = total_weight))
#     unet_seed_7_rmse_global = np.sqrt(np.average((ds_unet_seed_7_mean - ds_sp_1_mean) ** 2, weights = total_weight))
#     unet_seed_43_rmse_global = np.sqrt(np.average((ds_unet_seed_43_mean - ds_sp_1_mean) ** 2, weights = total_weight))
#     unet_seed_1024_rmse_global = np.sqrt(np.average((ds_unet_seed_1024_mean - ds_sp_1_mean) ** 2, weights = total_weight))
#     squeezeformer_seed_7_rmse_global = np.sqrt(np.average((ds_squeezeformer_seed_7_mean - ds_sp_1_mean) ** 2, weights = total_weight))
#     squeezeformer_seed_43_rmse_global = np.sqrt(np.average((ds_squeezeformer_seed_43_mean - ds_sp_1_mean) ** 2, weights = total_weight))
#     squeezeformer_seed_1024_rmse_global = np.sqrt(np.average((ds_squeezeformer_seed_1024_mean - ds_sp_1_mean) ** 2, weights = total_weight))

#     # Plot on the corresponding axis
#     line_mmf, = ax.plot(mmf_rmse, lev, label=f'{mmf_rmse_global:.2f}', linestyle='-.', color='black')
#     line_unet_seed_7, = ax.plot(unet_seed_7_rmse, lev, label=f'{unet_seed_7_rmse_global:.2f}', color='green', linestyle = '-.')
#     line_unet_seed_43, = ax.plot(unet_seed_43_rmse, lev, label=f'{unet_seed_43_rmse_global:.2f}', color='green', linestyle = '-.')
#     line_unet_seed_1024, = ax.plot(unet_seed_1024_rmse, lev, label=f'{unet_seed_1024_rmse_global:.2f}', color='green', linestyle = '-.')
#     line_squeezeformer_seed_7, = ax.plot(squeezeformer_seed_7_rmse, lev, label=f'{squeezeformer_seed_7_rmse_global:.2f}', color='purple', linestyle = '-.')
#     line_squeezeformer_seed_43, = ax.plot(squeezeformer_seed_43_rmse, lev, label=f'{squeezeformer_seed_43_rmse_global:.2f}', color='purple', linestyle = '-.')
#     line_squeezeformer_seed_1024, = ax.plot(squeezeformer_seed_1024_rmse, lev, label=f'{squeezeformer_seed_1024_rmse_global:.2f}', color='purple', linestyle = '-.')
    
#     ax.set_xlim(left=0)
#     ax.tick_params(axis='both', labelsize=12)
#     ax.set_title(title+f' ({units_dict[var]})', fontsize=14, loc='center')  # Add main title with subplot label
#     ax.set_xlabel(f'{units_dict[var]}', fontsize=14)  # Keep unit in x-label
#     ax.invert_yaxis()  # Reverse the y-axis
#     ax.legend(fontsize=8, ncol = 2)

# handles = [line_mmf, line_unet_seed_7, line_squeezeformer_seed_7]
# labels = ['MMF2', 'U-Net', 'Squeezeformer']

# fig.legend(handles, labels, loc='center right', bbox_to_anchor=(1.27, 0.5), title='Model')
# fig.suptitle('Five Year Global Mean Root Mean Squared Error')
# # Set a shared y-label for the first column
# axes[0].set_ylabel('Hybrid pressure (hPa)', fontsize=14)
# axes[3].set_ylabel('Hybrid pressure (hPa)', fontsize=14)
# plt.gca().invert_yaxis() 
# plt.tight_layout()
# # plt.savefig('state_rmse_profiles_and_scalar.pdf', format='pdf', dpi=400, bbox_inches='tight')
# plt.show()

In [37]:
standard_path = '/pscratch/sd/j/jerrylin/hugging/E3SM-MMF_ne4/online_runs/climsim3_ensembles_good/standard/five_year_runs/'

ds_unet_seed_7 = xr.open_mfdataset(os.path.join(standard_path, 'unet_seed_7/run/unet_seed_7.eam.h0.000[34567]*.nc'))

In [19]:
ds_sp_1_hourly = xr.open_mfdataset('/pscratch/sd/z/zeyuanhu/hu_etal2024_data_v2/data_hourly/precip_hourly/mmf_ref/PRECT-control_fullysp_jan_wmlio_r3.eam.h2*')

precip_dir = '/pscratch/sd/j/jerrylin/hugging/E3SM-MMF_ne4/online_runs/climsim3_ensembles_good/standard/five_year_runs/unet_seed_7/run/precip_dir'
test_load = xr.open_mfdataset(f'{precip_dir}/precip_unet_seed_7.eam.h2.000[34567]*.nc', 
                              compat = 'override', 
                              join = 'override', 
                              coords = 'minimal',
                              data_vars = ['PRECT', 'PRECC'])

In [74]:
sp_mean_precip = (ds_sp_1['PRECC'] + ds_sp_1['PRECL']).mean(dim = 'time').values[None,:]
unet_seed_7_mean_precip = (ds_unet_seed_7['PRECC'] + ds_unet_seed_7['PRECL']).mean(dim = 'time').values[None,:]

In [129]:
sp_zonal_mean_precip = data.zonal_bin_weight_2d(sp_mean_precip)
unet_seed_7_zonal_mean_precip = data.zonal_bin_weight_2d(unet_seed_7_mean_precip)
scaling = 86400 * 1000
land_frac = ds_sp_1['LANDFRAC'][2].values
ocean_frac = ds_sp_1['OCNFRAC'][2].values
land_area = data.grid_info_area * land_frac
ocean_area = data.grid_info_area * ocean_frac
data.lat_bin_land_area_sums = np.array([np.sum(land_area[data.lat_bin_dict[lat_bin]]) for lat_bin in data.lat_bin_dict.keys()])
data.lat_bin_land_area_divs = np.array([1/data.lat_bin_land_area_sums[bin_index] for bin_index in data.lat_bin_indices])
data.lat_bin_ocean_area_sums = np.array([np.sum(ocean_area[data.lat_bin_dict[lat_bin]]) for lat_bin in data.lat_bin_dict.keys()])
data.lat_bin_ocean_area_divs = np.array([1/data.lat_bin_ocean_area_sums[bin_index] for bin_index in data.lat_bin_indices])
data.lat_bin_land_area_divs[np.isnan(data.lat_bin_land_area_divs)] = 1
data.lat_bin_ocean_area_divs[np.isnan(data.lat_bin_ocean_area_divs)] = 1

/tmp/ipykernel_1831804/4294040189.py:11: RuntimeWarning: divide by zero encountered in scalar divide
  data.lat_bin_ocean_area_divs = np.array([1/data.lat_bin_ocean_area_sums[bin_index] for bin_index in data.lat_bin_indices])


In [128]:
type(data.lat_bin_dict[lat_bin])

NameError: name 'lat_bin' is not defined

In [121]:
ocean_area.shape

(384,)

In [125]:
data.lat_bin_dict.keys()

dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17])

In [122]:
data.grid_info_area.shape

(384,)

In [113]:
landfrac.shape

(384,)

In [117]:
ds_sp_1['LANDFRAC'][2].compute()

<xarray.DataArray 'LANDFRAC' (ncol: 384)>
array([0.        , 0.        , 0.29818895, 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.11482117,
       0.        , 0.6120708 , 0.1580556 , 0.        , 0.92459375,
       0.14176321, 0.6163086 , 0.        , 0.48771408, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.00412682,
       0.8759779 , 0.0312262 , 0.8928579 , 1.        , 0.5670488 ,
       1.        , 0.6075591 , 0.        , 0.        , 0.        ,
       0.        , 0.06457119, 0.5328895 , 0.48900962, 1.        ,
       0.5848698 , 1.        , 1.        , 1.        , 1.        ,
       0.96570027, 1.        , 0.96898264, 0.        , 0.        ,
       0.        , 0.        , 0.1771329 , 0.9636085 , 0.        ,
       0.6853744 , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 0.        ,
       0.        , 0.2577265 , 0.01043962, 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.26591393, 0.48098204, 0.7893597 , 1.        ,
       0.14610678, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.0058465 , 0.23369926, 0.05267295, 0.4628283 , 0.32188153,
       0.16571027, 0.21698871, 0.        , 0.48587114, 0.        ,
...
       0.        , 0.05593225, 0.        , 0.00187966, 0.        ,
       0.5290892 , 0.06375664, 0.54020256, 0.9150359 , 0.05830434,
       0.3331866 , 1.        , 0.79974276, 0.6406886 , 0.36277276,
       0.04199513, 0.01200373, 0.        , 0.03022684, 0.01827079,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.13075429, 0.02429591, 0.        ,
       0.31509352, 0.01191733, 0.10114167, 0.7464533 , 0.9840856 ,
       0.5089663 , 0.9142658 , 1.        , 1.        , 1.        ,
       1.        , 0.84086657, 0.34533718, 0.87955743, 0.36781865,
       0.64283264, 0.32629076, 0.39952868, 0.5609657 , 0.20446755,
       0.61555773, 0.05034225, 0.57173735, 1.        , 1.        ,
       1.        , 1.        , 1.        , 0.9284079 , 1.        ,
       1.        , 0.6668711 , 0.04826103, 0.84903616, 0.23192665,
       0.00116618, 0.781596  , 0.32519385, 0.92513454, 1.        ,
       1.        , 1.        , 1.        , 0.6466258 , 0.23441386,
       0.        , 0.        , 0.4500192 , 0.35226062, 0.01318306,
       0.02435855, 0.5253288 , 0.45897755, 0.        , 0.06355273,
       0.7433151 , 0.9840949 , 0.14168315, 0.24283944], dtype=float32)
Coordinates:
    time     object 0003-04-01 00:00:00
Dimensions without coordinates: ncol
Attributes:
    units:         1
    long_name:     Fraction of sfc area covered by land
    cell_methods:  time: mean

In [ ]:
# calculate zonal mean precipitation averaged globally, over ocean, and over land.
sp_tmean = (ds_sp['PRECC']+ds_sp['PRECL']).mean(dim=('time')).compute()
nn_tmean = (ds_nn['PRECC']+ds_nn['PRECL']).mean(dim=('time')).compute()
sp_zm, lats_sorted = zonal_mean_area_weighted(sp_tmean, grid_area, lat)
nn_zm, lats_sorted = zonal_mean_area_weighted(nn_tmean, grid_area, lat)
scaling = 86400*1000
land_frac = ds_sp.LANDFRAC[2].compute().values
ocean_frac = 1 - land_frac
sp_zm_land, lats_sorted = zonal_mean_area_weighted(sp_tmean, grid_area*land_frac, lat)
nn_zm_land, lats_sorted = zonal_mean_area_weighted(nn_tmean, grid_area*land_frac, lat)
sp_zm_ocean, lats_sorted = zonal_mean_area_weighted(sp_tmean, grid_area*ocean_frac, lat)
nn_zm_ocean, lats_sorted = zonal_mean_area_weighted(nn_tmean, grid_area*ocean_frac, lat)

data_sp = scaling*xr.DataArray(sp_zm, dims=["latitude"],
                                   coords={"latitude": lats_sorted})
data_nn = scaling*xr.DataArray(nn_zm, dims=["latitude"],
                                   coords={"latitude": lats_sorted})

data_sp_land = scaling*xr.DataArray(sp_zm_land, dims=["latitude"],
                                   coords={"latitude": lats_sorted})
data_nn_land = scaling*xr.DataArray(nn_zm_land, dims=["latitude"],
                                   coords={"latitude": lats_sorted})

data_sp_ocean = scaling*xr.DataArray(sp_zm_ocean, dims=["latitude"],
                                   coords={"latitude": lats_sorted})
data_nn_ocean = scaling*xr.DataArray(nn_zm_ocean, dims=["latitude"],
                                   coords={"latitude": lats_sorted})

In [ ]:
ds_sp_prect = xr.open_mfdataset(data_path+'data_hourly/precip_hourly/mmf_ref/PRECT*nc')
ds_nn_prect = xr.open_mfdataset(data_path+'data_hourly/precip_hourly/unet_v5/huber_rop/PRECT*nc')

data_sp_h2 = ds_sp_prect.PRECT.compute().values * 86400 * 1000  # Convert to mm/day
data_nn_h2 = ds_nn_prect.PRECT.compute().values * 86400 * 1000  # Convert to mm/day

In [ ]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt

land_frac = ds_sp.LANDFRAC[2].compute().values  # Example, replace with the actual land_frac array
ocean_frac = 1 - land_frac

# Flatten the data
data_sp_flat = data_sp_h2.flatten()
data_nn_flat = data_nn_h2.flatten()

# Repeat the grid_area to match the length of the flattened data
weights_sp = np.tile(grid_area, data_sp_h2.shape[0])
weights_nn = np.tile(grid_area, data_nn_h2.shape[0])

# Global weights
weights_sp_global = weights_sp
weights_nn_global = weights_nn

# Land weights
weights_sp_land = weights_sp * np.tile(land_frac, data_sp_h2.shape[0])
weights_nn_land = weights_nn * np.tile(land_frac, data_nn_h2.shape[0])

# Ocean weights
weights_sp_ocean = weights_sp * np.tile(ocean_frac, data_sp_h2.shape[0])
weights_nn_ocean = weights_nn * np.tile(ocean_frac, data_nn_h2.shape[0])

In [ ]:
# Define latitude ticks and labels
lat_ticks = [-60, -30, 0, 30, 60]
lat_labels = ['60S', '30S', '0', '30N', '60N']

# Create figure and axes
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(9, 3.7))

# First plot: Zonal mean precipitation
ax1 = axes[0]
# ax1.plot(lats_sorted, data_sp, label='MMF Global', color='blue', linestyle='-', marker='o')
# ax1.plot(lats_sorted, data_nn, label='NN Global', color='blue', linestyle='--', marker='x')
# ax1.plot(lats_sorted, data_sp_land, label='MMF Land', color='orange', linestyle='-', marker='o')
# ax1.plot(lats_sorted, data_nn_land, label='NN Land', color='orange', linestyle='--', marker='x')
# ax1.plot(lats_sorted, data_sp_ocean, label='MMF Ocean', color='red', linestyle='-', marker='o')
# ax1.plot(lats_sorted, data_nn_ocean, label='NN Ocean', color='red', linestyle='--', marker='x')
ax1.plot(lats_sorted, data_sp, label='MMF Global', color='blue', linestyle='-')# , marker='o')
ax1.plot(lats_sorted, data_nn, label='NN Global', color='blue', linestyle='--')# , marker='x')
ax1.plot(lats_sorted, data_sp_land, label='MMF Land', color='orange', linestyle='-')# , marker='o')
ax1.plot(lats_sorted, data_nn_land, label='NN Land', color='orange', linestyle='--')# , marker='x')
ax1.plot(lats_sorted, data_sp_ocean, label='MMF Ocean', color='red', linestyle='-')# , marker='o')
ax1.plot(lats_sorted, data_nn_ocean, label='NN Ocean', color='red', linestyle='--')# , marker='x')

ax1.set_xlabel('Latitude')
ax1.set_ylabel('Precipitation (mm/day)')
ax1.set_xticks(lat_ticks)
ax1.set_xticklabels(lat_labels)
ax1.set_title('(a) Mean Precipitation')
ax1.legend()
ax1.set_ylim(0, 8)
ax1.set_xlim(-90,90)
# Second plot: Weighted histogram of precipitation
ax2 = axes[1]
bins_lev = np.arange(-2,180,4)
def plot_histogram(ax, data_flat, weights, label, color, linestyle):
    hist, bins = np.histogram(data_flat, bins=bins_lev, weights=weights, density=True)
    bin_centers = (bins[:-1] + bins[1:]) / 2
    ax.plot(bin_centers, hist, label=label, color=color, linestyle=linestyle, linewidth=2)

plot_histogram(ax2, data_sp_flat, weights_sp_global, 'MMF Global', 'blue', '-')
plot_histogram(ax2, data_nn_flat, weights_nn_global, 'NN Global', 'blue', '--')
plot_histogram(ax2, data_sp_flat, weights_sp_land, 'MMF Land', 'orange', '-')
plot_histogram(ax2, data_nn_flat, weights_nn_land, 'NN Land', 'orange', '--')
plot_histogram(ax2, data_sp_flat, weights_sp_ocean, 'MMF Ocean', 'red', '-')
plot_histogram(ax2, data_nn_flat, weights_nn_ocean, 'NN Ocean', 'red', '--')

ax2.set_yscale('log')
ax2.set_xlabel('Precipitation (mm/day)')
ax2.set_ylabel('Frequency')
ax2.set_title('(b) Histogram of Precipitation')
ax2.set_ylim(1e-8,0.5)
ax2.set_xlim(0,180)

# Adjust layout
# plt.tight_layout()
plt.subplots_adjust(wspace=0.3)  # Adjust the width space between subplots
# Show plot
# plt.savefig('precipitation_distribution_hist_nopruning_noclass.eps', format='eps', dpi=600)
plt.show()